In [29]:
#导入初始化
import tecplot as tp
import numpy as np
import pandas as pd
from tecplot.constant import PlotType,SliceSurface,ContLegendLabelLocation
tp.session.connect()
frame = tp.active_frame()
dataset = frame.dataset
fmaps = frame.plot().fieldmaps([dataset.zone(i) for i in range(frame.plot().num_fieldmaps)])
fmaps.scatter.show = False

#坐标转换
def axes_transform(frame = tp.active_frame(),transform = {'z':'y','y':'z'},init = True):
    axes = frame.plot().axes #笛卡尔三维坐标系
    plot = frame.plot()
    for i in transform:
        if i == 'x' or i == 'X':
            axes.x_axis.variable = dataset.variable(f'{transform[i]}')
        elif i == 'y' or i == 'Y':
            axes.y_axis.variable = dataset.variable(f'{transform[i]}')
        elif i == 'z' or i == 'Z':
            axes.z_axis.variable = dataset.variable(f'{transform[i]}')
    if init: #初始化坐标轴，重设缩放
        axes.reset_range()
        axes.reset_scale()
        plot.view.fit()

#求解速度场
def velocity_field():
    '''
    数据转换函数，使用方式参考：
    二氧化碳浓度场：
    tp.data.operate.execute_equation('{CO2 (ppm)} = {Mass fraction of co2}*1E6')
    '''
    tp.data.operate.execute_equation('{Velocity (m/s)} = SQRT({X Velocity}**2 + {Y Velocity}**2 + {Z Velocity}**2)')

#创建剖切面云图
def creat_slice(frame = tp.active_frame(),variable = 'Static Pressure',
                translucency = True,shade = False,slices = True):
    
    plot = frame.plot()
    plot.use_translucency = translucency
    plot.contour(0).variable = dataset.variable(variable)
    plot.show_shade = shade
    plot.show_slices = slices
    plot.slice(0).orientation = SliceSurface.XPlanes
    plot.slice(1).orientation = SliceSurface.YPlanes
    plot.slice(2).orientation = SliceSurface.ZPlanes
    slices = plot.slices(0,1,2)
    slices.show = True
    slices.edge.show = True
    slices.edge.line_thickness = 0.25
    slices.effects.use_translucency = True
    return slices

#重置剖切面位置
def reset_slices(plot = tp.active_frame().plot()):
    axes = plot.axes
    l_axes = [axes.x_axis,axes.y_axis,axes.z_axis]
    slices = creat_slice()
    temp = ()
    j = 0
    for each in axes:
        temp = temp + ((each.max + each.min)/2,)
        init_origin = [0,0,0]
        init_origin[j] = temp[j]
        slices[j].origin = tuple(init_origin)
        j += 1

#云图图例格式化
def legend_init(frame = tp.active_frame(),fonts = 'Time New Roman'):
    legend = frame.plot().contour(0).legend
    legend.show = True
    legend.vertical = False
    legend.label_location = ContLegendLabelLocation.ContourLevels
    legend.position = (100,100)
    legend.box.box_type = tp.constant.TextBox.None_
    legend.number_font.typeface = fonts
    legend.number_font.size = 2.5
    legend.header.font.typeface = fonts
    legend.header.font.size = 2.5

#方向标格式化
def axes_init(frame = tp.active_frame()):
    plot = frame.plot()
    plot.axes.orientation_axis.show = True
    plot.axes.orientation_axis.position = 7,7
    plot.axes.orientation_axis.size = 7

#单个方向剖面图重置
def reset_one_slice(direction = 'x'):
    slices = tp.active_frame().plot().slices(0,1,2)
    direction_dict = {'x':0,'y':1,'z':2}
    vec = direction_dict[direction]
    axes = tp.active_frame().plot().axes
    l_axes = [axes.x_axis,axes.y_axis,axes.z_axis]
    axes_range = (l_axes[vec].max + l_axes[vec].min)/2
    init_origin = [0,0,0]
    init_origin[vec] = axes_range
    slices[vec].origin = tuple(init_origin)

#各方向自定义云图
def modify_slice(direction = 'x',num = 0):
    direciton_dict = {'x':0,'y':1,'z':2}
    vec = direciton_dict[direction]
    frame = tp.active_frame()
    plot = frame.plot()
    axes = plot.axes
    slices = plot.slices(0,1,2) #0,1,2分别对应x,y,z方向
    slices[vec].show = True
    init_slices = [0,0,0]
    init_slices[vec] = num
    slices[vec].origin = tuple(init_slices)
    l_axes = [axes.x_axis,axes.y_axis,axes.z_axis]
    if num > l_axes[0].max or num < l_axes[0].min:
        reset_one_slice('x')
        print('Range: %.2f to %.2f' %(l_axes[0].min,l_axes[0].max))
        print('Now number: %.2f  Out of range of coordinates !' %(num))

#去除数据点集
def clear_extract(name = 'line'):
    while True:
        try:
            dataset.delete_zones(dataset.zone(name))
        except:
            print('complete')
            break

Connecting to Tecplot 360 TecUtil Server on:
    tcp://localhost:7600
Connection established.


In [31]:
# 线数据提取
plot = frame.plot()
position_origin = [1.67,-3.8,1.67] # 该处Y、Z坐标与Probe中的Y、Z坐标相反
position_matrix = np.arange(position_origin[0],position_origin[0]+3,1)
position_res = []
plot.show_scatter = True
for i in position_matrix:
    position_origin[0] = i
    position_res.append(position_origin.copy()) # 需要使用copy()函数，否则position_origin会被修改
line = tp.data.extract.extract_line(position_res)
variables = line.values('Static Pressure')
pd_charts = pd.DataFrame(columns=['Coordinate Point','Static Pressure'])
for i in range(len(position_res)):
    pd_charts.loc[i] = [position_res[i],variables[i]]
line.name = 'line'
clear_extract(line.name)
clear_extract('Extracted Points')

complete
complete


'Extracted Points'

complete


In [83]:
print(dataset.zone_names)

['human_fluid', 'wall_sides:12082', 'wall_ground:12085', 'wall_top:12086', 'wall_people:12087', 'inlet_wall:12084', 'inlet_nose:12088', 'outlet_wall:12083', 'inlet_nose', 'wall_people', 'wall_top', 'wall_ground', 'inlet_wall', 'outlet_wall', 'wall_sides', 'line', 'line', 'Extracted Points', 'line', 'line', 'Extracted Points', 'Extracted Points']


In [56]:
def direciton_vec(direction = 'x'):
    direciton_dict = {'x':0,'y':1,'z':2}
    if type(direction) == str:
        vec = direciton_dict[direction]
        slices = tp.active_frame().plot().slices(0,1,2)
        return slices[vec]
    elif type(direction) == list:
        res = []
        for each in direction:
            res.append(direciton_vec(each))
            return res
direciton_vec(['x','y','z'])

In [78]:
# 初始化步骤
axes_transform()
velocity_field()
creat_slice()
legend_init()
axes_init()